In [44]:
import re
from os import listdir
from os.path import isfile, join
import random
import json

In [67]:
def split_script_into_json(src, train_dest, test_dest, unique_id, block_count, test_count):
    with open(src, "r") as f:
        lines = f.readlines()
        line_count = len(lines)
        lines_per_block = line_count // block_count
        blocks = random.sample(range(0, block_count), test_count)
        train_data = ""
        test_data = {}
        test_itr = 1
        for i in range(block_count):
            start_index = i*lines_per_block
            end_index = start_index + lines_per_block
            split_index = (start_index + end_index) // 2
            
            if i in blocks:
                test_data["test"+str(test_itr)+"_input"] = ''.join(lines[start_index:split_index])
                test_data["test"+str(test_itr)+"_true"]  = ''.join(lines[split_index+1:end_index])
                test_data["test"+str(test_itr)+"_id"]    = unique_id + test_itr - 1
                test_itr += 1
            else:
                train_data += ''.join(lines[start_index:split_index])
        
        test_dest = test_dest[:-4] + ".json"
        with open(test_dest, 'w+') as outfile:
            json.dump(test_data, outfile)
            
        with open(train_dest, "w+") as f:
            f.write(train_data)
            

def generate_test_jsons_entire_dir(srcpath, train_destpath, test_destpath):
    unique_id = 0
    blocks_per_file = 20
    tests_per_file = 3
    for f in listdir(srcpath):
        src = join(srcpath, f)
        dest = join(destpath, f)
        if isfile(src):
            split_script_into_json(src, train_dest, test_dest, unique_id, blocks_per_file, tests_per_file)
        unique_id += tests_per_file
    
def parse_script(src, dest):
    with open(src, "r") as f:
        parsed = ""
        added_new = True # Used for ignoring consequtive empty lines
        for line in f:
            temp = line.strip(" ")
            if temp != "\n":
                temp = temp.strip("\n")
                if (re.search('[a-zA-Z]', temp)): # Ignore page number lines
                    parsed += line.strip() + " "
                    added_new = False
            else:
                if not added_new:
                    parsed += "\n"
                    added_new = True
                continue
    with open(dest, "w") as f:
        f.write(parsed)

def process_entire_dir(srcpath, destpath):
    for f in listdir(srcpath):
        src = join(srcpath, f)
        dest = join(destpath, f)
        if isfile(src):
            parse_script(src, dest)
    print("Processed: {} scripts".format(len(listdir(srcpath))))

In [ ]:
process_entire_dir("./raw_data/", "./parsed_data/")

In [68]:
generate_test_jsons_entire_dir("./parsed_data/", "./training_cases/", "./test_cases/")

NameError: name 'destpath' is not defined